<a href="https://colab.research.google.com/github/eckoecho/CodingDojo/blob/EDA/Column_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", 100)
import missingno as msno
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config
set_config(transform_output="pandas")

In [102]:
#Load data
fpath="/content/drive/MyDrive/CodingDojo/02-MachineLearning/Week05/Data/ames-housing-dojo-for-ml.csv"
df = pd.read_csv(fpath)
df = df.set_index("PID")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2930 entries, 907227090 to 902201120
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MS Zoning         2930 non-null   object 
 1   Lot Frontage      2440 non-null   float64
 2   Lot Area          2930 non-null   int64  
 3   Street            2930 non-null   object 
 4   Alley             198 non-null    object 
 5   Utilities         2930 non-null   object 
 6   Neighborhood      2930 non-null   object 
 7   Bldg Type         2930 non-null   object 
 8   House Style       2930 non-null   object 
 9   Overall Qual      2930 non-null   int64  
 10  Overall Cond      2930 non-null   int64  
 11  Year Built        2930 non-null   int64  
 12  Year Remodeled    2930 non-null   int64  
 13  Exter Qual        2930 non-null   object 
 14  Exter Cond        2930 non-null   object 
 15  Bsmt Unf Sqft     2929 non-null   float64
 16  Total Bsmnt Sqft  2929 non-nu

,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Utilities,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remodeled,Exter Qual,Exter Cond,Bsmt Unf Sqft,Total Bsmnt Sqft,Central Air,Living Area Sqft,Bedroom,Kitchen,Total Rooms,Garage Type,Garage Yr Blt,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Fence,SalePrice,Month,Year,Total Full Baths,Total Half Baths
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
907227090,RL,60.0,7200,Pave,NaN,AllPub,CollgCr,1Fam,1Story,5,8,1972,1972,TA,TA,427.0,864.0,Y,864.0,3,1,5,Detchd,1977.0,1.0,297.0,TA,TA,Y,MnPrv,119900.0,3,2006,1.0,0.0
527108010,RL,134.0,19378,Pave,NaN,AllPub,Gilbert,1Fam,2Story,7,5,2005,2006,Gd,TA,1335.0,1392.0,Y,2462.0,4,1,9,Attchd,2006.0,2.0,576.0,TA,TA,Y,NaN,320000.0,3,2006,3.0,1.0
534275170,RL,NaN,12772,Pave,NaN,AllPub,NAmes,1Fam,1Story,6,8,1960,1998,TA,Gd,460.0,958.0,Y,958.0,2,1,5,Attchd,1960.0,1.0,301.0,TA,TA,Y,NaN,151500.0,4,2007,1.0,0.0
528104050,RL,114.0,14803,Pave,NaN,AllPub,NridgHt,1Fam,1Story,10,5,2007,2008,Ex,TA,442.0,2078.0,Y,2084.0,2,1,7,Attchd,2007.0,3.0,1220.0,TA,TA,Y,NaN,385000.0,6,2008,3.0,0.0
533206070,FV,32.0,3784,Pave,Pave,AllPub,Somerst,TwnhsE,1Story,8,5,2006,2007,Gd,TA,1451.0,1511.0,Y,1565.0,2,1,5,Attchd,2006.0,2.0,476.0,TA,TA,Y,NaN,193800.0,2,2007,3.0,0.0


# Train Test Split

In [103]:
#Set features and target
X= df.drop(columns="SalePrice")
y= df["SalePrice"]
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
X_train.head()

,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Utilities,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remodeled,Exter Qual,Exter Cond,Bsmt Unf Sqft,Total Bsmnt Sqft,Central Air,Living Area Sqft,Bedroom,Kitchen,Total Rooms,Garage Type,Garage Yr Blt,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Fence,Month,Year,Total Full Baths,Total Half Baths
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
905475520,RL,NaN,11515,Pave,NaN,AllPub,Edwards,1Fam,1Story,4,5,1958,1994,TA,TA,0.0,0.0,N,943.0,3,1,5,Detchd,1958.0,1.0,308.0,TA,TA,N,NaN,8,2007,1.0,0.0
909254010,RL,53.0,7155,Pave,NaN,AllPub,Crawfor,1Fam,2Story,7,8,1918,1990,TA,TA,600.0,600.0,Y,1228.0,3,1,6,Detchd,1918.0,1.0,215.0,Fa,TA,Y,MnPrv,5,2009,1.0,0.0
531450090,RL,NaN,7153,Pave,NaN,AllPub,SawyerW,1Fam,1Story,6,5,1991,1991,TA,TA,78.0,1278.0,Y,1294.0,3,1,6,Attchd,1991.0,2.0,496.0,TA,TA,Y,GdWo,6,2008,3.0,0.0
903400040,RL,60.0,13515,Pave,Pave,AllPub,BrkSide,1Fam,2Story,6,6,1919,1950,TA,TA,764.0,764.0,Y,1824.0,3,1,8,Detchd,1940.0,2.0,520.0,TA,TA,N,GdPrv,7,2007,1.0,0.0
527107130,RL,60.0,8400,Pave,NaN,AllPub,Gilbert,1Fam,SLvl,7,5,1996,1997,TA,TA,100.0,384.0,Y,1628.0,3,1,7,BuiltIn,1996.0,2.0,390.0,TA,TA,Y,NaN,6,2009,2.0,1.0


#Create Pipelines and Define Tuples

##Numeric Pipeline

In [104]:
# PREPROCESSING PIPELINE FOR NUMERIC DATA
# Save list of column names
num_cols = X_train.select_dtypes("number").columns
print("Numeric Columns:", num_cols)
# instantiate preprocessors
impute_median = SimpleImputer(strategy='median')
scaler = StandardScaler()
# Make a numeric preprocessing pipeline
num_pipe = make_pipeline(impute_median, scaler)
num_pipe

Numeric Columns: Index(['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond',
       'Year Built', 'Year Remodeled', 'Bsmt Unf Sqft', 'Total Bsmnt Sqft',
       'Living Area Sqft', 'Bedroom', 'Kitchen', 'Total Rooms',
       'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Month', 'Year',
       'Total Full Baths', 'Total Half Baths'],
      dtype='object')


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

### Define Numeric Tuple for ColumnTransformer

In [105]:
# Making a numeric tuple for ColumnTransformer
num_tuple = ('numeric', num_pipe, num_cols)
num_tuple

('numeric',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                 ('standardscaler', StandardScaler())]),
 Index(['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond',
        'Year Built', 'Year Remodeled', 'Bsmt Unf Sqft', 'Total Bsmnt Sqft',
        'Living Area Sqft', 'Bedroom', 'Kitchen', 'Total Rooms',
        'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Month', 'Year',
        'Total Full Baths', 'Total Half Baths'],
       dtype='object'))

## Ordinal Pipeline

In [106]:
# PREPROCESSING PIPELINE FOR ORDINAL DATA
# Save list of column names
ord_cols = ['Exter Qual','Exter Cond', 'Garage Qual',"Garage Cond"]
print("Ordinal Columns:", ord_cols)
# Create imputer for ordinal data 
impute_na_ord = SimpleImputer(strategy='constant', fill_value='NA')
## Making the OrdinalEncoder
# Specifying order of categories for our  Ordinal Qual/Cond Columms
qual_cond_order = ['NA','Po', 'Fa', 'TA', 'Gd', 'Ex']
# Making the list of order lists for OrdinalEncoder
ordinal_category_orders = [qual_cond_order, qual_cond_order,
                           qual_cond_order, qual_cond_order]                           
ord_encoder = OrdinalEncoder(categories=ordinal_category_orders)
# Making a final scaler to scale category #'s
scaler_ord = StandardScaler()
## Making an ord_pipe 
ord_pipe = make_pipeline(impute_na_ord, ord_encoder, scaler_ord)
ord_pipe

Ordinal Columns: ['Exter Qual', 'Exter Cond', 'Garage Qual', 'Garage Cond']


Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='NA', strategy='constant')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['NA', 'Po', 'Fa', 'TA', 'Gd',
                                             'Ex'],
                                            ['NA', 'Po', 'Fa', 'TA', 'Gd',
                                             'Ex'],
                                            ['NA', 'Po', 'Fa', 'TA', 'Gd',
                                             'Ex'],
                                            ['NA', 'Po', 'Fa', 'TA', 'Gd',
                                             'Ex']])),
                ('standardscaler', StandardScaler())])

### Define Ordinal Tuple for ColumnTransformer

In [107]:
# Making an ordinal_tuple for ColumnTransformer
ord_tuple = ('ordinal', ord_pipe, ord_cols)
ord_tuple

('ordinal',
 Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value='NA', strategy='constant')),
                 ('ordinalencoder',
                  OrdinalEncoder(categories=[['NA', 'Po', 'Fa', 'TA', 'Gd',
                                              'Ex'],
                                             ['NA', 'Po', 'Fa', 'TA', 'Gd',
                                              'Ex'],
                                             ['NA', 'Po', 'Fa', 'TA', 'Gd',
                                              'Ex'],
                                             ['NA', 'Po', 'Fa', 'TA', 'Gd',
                                              'Ex']])),
                 ('standardscaler', StandardScaler())]),
 ['Exter Qual', 'Exter Cond', 'Garage Qual', 'Garage Cond'])

## Categorical Pipeline



In [108]:
# PREPROCESSING PIPELINE FOR ONE-HOT-ENCODED DATA
# Save list of column names
ohe_cols = X_train.select_dtypes('object').drop(columns=ord_cols).columns
print("OneHotEncoder Columns:", ohe_cols)
# Instantiate the individual preprocessors
impute_na = SimpleImputer(strategy='constant', fill_value = "NA")
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Make pipeline with imputer and encoder
ohe_pipe = make_pipeline(impute_na, ohe_encoder)
ohe_pipe

OneHotEncoder Columns: Index(['MS Zoning', 'Street', 'Alley', 'Utilities', 'Neighborhood',
       'Bldg Type', 'House Style', 'Central Air', 'Garage Type', 'Paved Drive',
       'Fence'],
      dtype='object')


Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='NA', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

### Define Categorical Tuple for ColumnTransformer

In [109]:
# Making a ohe_tuple for ColumnTransformer
ohe_tuple = ('categorical', ohe_pipe, ohe_cols)
ohe_tuple

('categorical',
 Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value='NA', strategy='constant')),
                 ('onehotencoder',
                  OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
 Index(['MS Zoning', 'Street', 'Alley', 'Utilities', 'Neighborhood',
        'Bldg Type', 'House Style', 'Central Air', 'Garage Type', 'Paved Drive',
        'Fence'],
       dtype='object'))

#Instantiate the ColumnTransformer

In [110]:
# Instantiate with verbose_feature_names_out=False
col_transformer = ColumnTransformer([num_tuple, ord_tuple, ohe_tuple],
                                    verbose_feature_names_out=False)
col_transformer

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond',
       'Year Built', 'Year Remodeled', 'Bsmt Unf Sqft', 'Total Bsmnt Sqft',
       'Living Area Sqft', 'Bedroom', 'Kitchen', 'Total Rooms',
       'Garage Yr Blt', 'Garage Cars', 'Garage...
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['MS Zoning', 'Street', 'Alley', 'Utilities', 'Neighborhood',
       'Bldg Type', 'House Style', 'Central Air', 'Garage Type', 'Paved Drive',
       'Fence'],
      dtype='object'))],
                  verbose_feature_names_out=False)

# Fit the Transformer on Training Data Only


In [111]:
# Fit on training data
col_transformer.fit(X_train)

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond',
       'Year Built', 'Year Remodeled', 'Bsmt Unf Sqft', 'Total Bsmnt Sqft',
       'Living Area Sqft', 'Bedroom', 'Kitchen', 'Total Rooms',
       'Garage Yr Blt', 'Garage Cars', 'Garage...
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['MS Zoning', 'Street', 'Alley', 'Utilities', 'Neighborhood',
       'Bldg Type', 'House Style', 'Central Air', 'Garage Type', 'Paved Drive',
       'Fence'],
      dtype='object'))],
                  verbose_feature_names_out=False)

# Transform Training and Test data

In [112]:
# Transform the training data
X_train_processed = col_transformer.transform(X_train)
# Transform the test data
X_test_processed = col_transformer.transform(X_test)
X_train_processed.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remodeled,Bsmt Unf Sqft,Total Bsmnt Sqft,Living Area Sqft,Bedroom,Kitchen,Total Rooms,Garage Yr Blt,Garage Cars,Garage Area,Month,Year,Total Full Baths,Total Half Baths,Exter Qual,Exter Cond,Garage Qual,Garage Cond,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_NA,Alley_Pave,Utilities_AllPub,Utilities_NoSewr,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Central Air_N,Central Air_Y,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_NA,Paved Drive_N,Paved Drive_P,Paved Drive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NA
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
905475520,-0.041873,0.208863,-1.453681,-0.502192,-0.423561,0.472590,-1.282165,-2.329907,-1.086450,0.183403,-0.203499,-0.909803,-0.820130,-0.989438,-0.743628,0.674753,-0.605551,-1.289301,-0.807982,-0.679141,-0.227105,0.282215,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
909254010,-0.750990,-0.413530,0.648845,2.193205,-1.738890,0.282376,0.093593,-0.996395,-0.525568,0.183403,-0.203499,-0.275453,-2.455923,-0.989438,-1.169681,-0.439858,0.908844,-1.289301,-0.807982,-0.679141,-0.227105,-1.092567,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
531450090,-0.041873,-0.413816,-0.051997,-0.502192,0.661586,0.329930,-1.103317,0.510475,-0.395679,0.183403,-0.203499,-0.275453,0.529399,0.313016,0.117642,-0.068321,0.151646,1.343221,-0.807982,-0.679141,-0.227105,0.282215,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
903400040,-0.420069,0.494365,-0.051997,0.396274,-1.706007,-1.619758,0.469633,-0.631901,0.647364,0.183403,-0.203499,0.993247,-1.556237,0.313016,0.227591,0.303216,-0.605551,-1.289301,-0.807982,-0.679141,-0.227105,0.282215,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
527107130,-0.420069,-0.235806,0.648845,-0.502192,0.826002,0.615250,-1.052872,-1.476459,0.261635,0.183403,-0.203499,0.358897,0.733873,0.313016,-0.367968,-0.068321,0.908844,0.026960,1.050800,-0.679141,-0.227105,0.282215,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0

# Include remainder = 'passthrough'

In [113]:
col_transformer = ColumnTransformer([num_tuple, ord_tuple], remainder="passthrough",
                                    verbose_feature_names_out=False)
col_transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond',
       'Year Built', 'Year Remodeled', 'Bsmt Unf Sqft', 'Total Bsmnt Sqft',
       'Living Area Sqft', 'Bedroom', 'Kitchen', 'Total Rooms',
       'Garage Yr B...
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['NA',
                                                                              'Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
                                                                             ['NA',
                                                                              'Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
                                                                             ['NA',
                                                                              'Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
                                                                             ['NA',
                                                                              'Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Exter Qual', 'Exter Cond', 'Garage Qual',
                                  'Garage Cond'])],
                  verbose_feature_names_out=False)

##Fit Transformer

In [114]:
# Fit on the training data
col_transformer.fit(X_train)
# Transorm the training data
X_train_processed_demo = col_transformer.transform(X_train)
# Transorm the test data
X_test_processed_demo = col_transformer.transform(X_test)
X_train_processed_demo.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remodeled,Bsmt Unf Sqft,Total Bsmnt Sqft,Living Area Sqft,Bedroom,Kitchen,Total Rooms,Garage Yr Blt,Garage Cars,Garage Area,Month,Year,Total Full Baths,Total Half Baths,Exter Qual,Exter Cond,Garage Qual,Garage Cond,MS Zoning,Street,Alley,Utilities,Neighborhood,Bldg Type,House Style,Central Air,Garage Type,Paved Drive,Fence
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
905475520,-0.041873,0.208863,-1.453681,-0.502192,-0.423561,0.472590,-1.282165,-2.329907,-1.086450,0.183403,-0.203499,-0.909803,-0.820130,-0.989438,-0.743628,0.674753,-0.605551,-1.289301,-0.807982,-0.679141,-0.227105,0.282215,0.272525,RL,Pave,NaN,AllPub,Edwards,1Fam,1Story,N,Detchd,N,NaN
909254010,-0.750990,-0.413530,0.648845,2.193205,-1.738890,0.282376,0.093593,-0.996395,-0.525568,0.183403,-0.203499,-0.275453,-2.455923,-0.989438,-1.169681,-0.439858,0.908844,-1.289301,-0.807982,-0.679141,-0.227105,-1.092567,0.272525,RL,Pave,NaN,AllPub,Crawfor,1Fam,2Story,Y,Detchd,Y,MnPrv
531450090,-0.041873,-0.413816,-0.051997,-0.502192,0.661586,0.329930,-1.103317,0.510475,-0.395679,0.183403,-0.203499,-0.275453,0.529399,0.313016,0.117642,-0.068321,0.151646,1.343221,-0.807982,-0.679141,-0.227105,0.282215,0.272525,RL,Pave,NaN,AllPub,SawyerW,1Fam,1Story,Y,Attchd,Y,GdWo
903400040,-0.420069,0.494365,-0.051997,0.396274,-1.706007,-1.619758,0.469633,-0.631901,0.647364,0.183403,-0.203499,0.993247,-1.556237,0.313016,0.227591,0.303216,-0.605551,-1.289301,-0.807982,-0.679141,-0.227105,0.282215,0.272525,RL,Pave,Pave,AllPub,BrkSide,1Fam,2Story,Y,Detchd,N,GdPrv
527107130,-0.420069,-0.235806,0.648845,-0.502192,0.826002,0.615250,-1.052872,-1.476459,0.261635,0.183403,-0.203499,0.358897,0.733873,0.313016,-0.367968,-0.068321,0.908844,0.026960,1.050800,-0.679141,-0.227105,0.282215,0.272525,RL,Pave,NaN,AllPub,Gilbert,1Fam,SLvl,Y,BuiltIn,Y,NaN


#Tranforming using 1 Transformer

In [116]:
# Making a tuple for ColumnTransformer with a single transformer (no pipeline)
impute_median = SimpleImputer(strategy='median')
# Create the tuple without adding a pipeline:
#           (choose_a_name, pipeline_or_transformer, list_of_columns)
num_impute_tuple = ('number-imputed', impute_median, num_cols)
num_impute_tuple

('number-imputed',
 SimpleImputer(strategy='median'),
 Index(['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond',
        'Year Built', 'Year Remodeled', 'Bsmt Unf Sqft', 'Total Bsmnt Sqft',
        'Living Area Sqft', 'Bedroom', 'Kitchen', 'Total Rooms',
        'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Month', 'Year',
        'Total Full Baths', 'Total Half Baths'],
       dtype='object'))

In [118]:
# Instantiate with verbose_feature_names_out=False
col_transformer = ColumnTransformer([num_impute_tuple, ord_tuple, ohe_tuple],
                                    verbose_feature_names_out=False)
col_transformer

ColumnTransformer(transformers=[('number-imputed',
                                 SimpleImputer(strategy='median'),
                                 Index(['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond',
       'Year Built', 'Year Remodeled', 'Bsmt Unf Sqft', 'Total Bsmnt Sqft',
       'Living Area Sqft', 'Bedroom', 'Kitchen', 'Total Rooms',
       'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Month', 'Year',
       'Total Full Baths', 'Total Half Baths'],
      dtype='ob...
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['MS Zoning', 'Street', 'Alley', 'Utilities', 'Neighborhood',
       'Bldg Type', 'House Style', 'Central Air', 'Garage Type', 'Paved Drive',
       'Fence'],
      dtype='object'))],
                  verbose_feature_names_out=False)

In [120]:
# Fit on the training data
col_transformer.fit(X_train)
# Transform the training  data
X_train_processed_demo = col_transformer.transform(X_train)
# Transform the test  data
X_test_processed_demo = col_transformer.transform(X_test)
# view results
X_train_processed_demo.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remodeled,Bsmt Unf Sqft,Total Bsmnt Sqft,Living Area Sqft,Bedroom,Kitchen,Total Rooms,Garage Yr Blt,Garage Cars,Garage Area,Month,Year,Total Full Baths,Total Half Baths,Exter Qual,Exter Cond,Garage Qual,Garage Cond,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_NA,Alley_Pave,Utilities_AllPub,Utilities_NoSewr,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Central Air_N,Central Air_Y,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_NA,Paved Drive_N,Paved Drive_P,Paved Drive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NA
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
905475520,68.0,11515.0,4.0,5.0,1958.0,1994.0,0.0,0.0,943.0,3.0,1.0,5.0,1958.0,1.0,308.0,8.0,2007.0,1.0,0.0,-0.679141,-0.227105,0.282215,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
909254010,53.0,7155.0,7.0,8.0,1918.0,1990.0,600.0,600.0,1228.0,3.0,1.0,6.0,1918.0,1.0,215.0,5.0,2009.0,1.0,0.0,-0.679141,-0.227105,-1.092567,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
531450090,68.0,7153.0,6.0,5.0,1991.0,1991.0,78.0,1278.0,1294.0,3.0,1.0,6.0,1991.0,2.0,496.0,6.0,2008.0,3.0,0.0,-0.679141,-0.227105,0.282215,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
903400040,60.0,13515.0,6.0,6.0,1919.0,1950.0,764.0,764.0,1824.0,3.0,1.0,8.0,1940.0,2.0,520.0,7.0,2007.0,1.0,0.0,-0.679141,-0.227105,0.282215,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
527107130,60.0,8400.0,7.0,5.0,1996.0,1997.0,100.0,384.0,1628.0,3.0,1.0,7.0,1996.0,2.0,390.0,6.0,2009.0,2.0,1.0,-0.679141,-0.227105,0.282215,0.272525,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [121]:
# Checking for null values
X_train_processed.isna().sum().sum()

0